# LOAN STATUS PREDICTION

### Submitted By
# MADDURI UMA   - D22025
# KENWIN DASS C - D22051

## Problem Statement: 
### This is a Binary classification problem in which we need to classify whether the loan will be approved or not and we need to predict our Target label which is “Loan Status”

### Importing Required Libraries

In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

#For train-test split
from sklearn.model_selection import train_test_split 

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

#Import label encoder
from sklearn import preprocessing

from sklearn import metrics
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,classification_report

from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
from collections import Counter


# TRAIN DATA

In [101]:
import os
os.getcwd()

'C:\\Users\\MY PC\\Downloads'

In [102]:
loans = pd.read_csv(r'C:\Users\MY PC\Desktop\PPP\Loans - Training Set.csv')
loans.head()

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,6cf51492-02a2-423e-b93d-676f05b9ad53,7c202b37-2add-44e8-9aea-d5b119aea935,Charged Off,12232,Short Term,7280.0,< 1 year,Rent,46643.0,Debt Consolidation,777.39,18.0,10.0,12,0,6762,7946,0.0,0.0
1,552e7ade-4292-4354-9ff9-c48031697d72,e7217b0a-07ac-47dd-b379-577b5a35b7c6,Charged Off,25014,Long Term,7330.0,10+ years,Home Mortgage,81099.0,Debt Consolidation,892.09,26.7,NaN,14,0,35706,77961,0.0,0.0
2,9b5e32b3-8d76-4801-afc8-d729d5a2e6b9,0a62fc41-16c8-40b5-92ff-9e4b763ce714,Charged Off,16117,Short Term,7240.0,9 years,Home Mortgage,60438.0,Home Improvements,1244.02,16.7,32.0,11,1,11275,14815,1.0,0.0
3,5419b7c7-ac11-4be2-a8a7-b131fb6d6dbe,30f36c59-5182-4482-8bbb-5b736849ae43,Charged Off,11716,Short Term,7400.0,3 years,Rent,34171.0,Debt Consolidation,990.94,10.0,NaN,21,0,7009,43533,0.0,0.0
4,1450910f-9495-4fc9-afaf-9bdf4b9821df,70c26012-bba5-42c0-8dcb-75295ada31bb,Charged Off,9789,Long Term,6860.0,10+ years,Home Mortgage,47003.0,Home Improvements,503.71,16.7,25.0,13,1,16913,19553,1.0,0.0


In [103]:
# data type
loans.dtypes

Loan ID                          object
Customer ID                      object
Loan Status                      object
Current Loan Amount               int64
Term                             object
Credit Score                    float64
Years in current job             object
Home Ownership                   object
Annual Income                   float64
Purpose                          object
Monthly Debt                     object
Years of Credit History         float64
Months since last delinquent    float64
Number of Open Accounts           int64
Number of Credit Problems         int64
Current Credit Balance            int64
Maximum Open Credit              object
Bankruptcies                    float64
Tax Liens                       float64
dtype: object

In [104]:
loans.shape

(111107, 19)

In [105]:
loans.describe()

,Current Loan Amount,Credit Score,Annual Income,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Bankruptcies,Tax Liens
count,1.111070e+05,89769.000000,8.976900e+04,111107.000000,52104.000000,111107.000000,111107.000000,1.111070e+05,110879.00000,111096.000000
mean,1.147718e+07,1075.795029,7.248600e+04,18.201003,34.902272,11.125105,0.168027,1.548314e+04,0.11766,0.029353
std,3.185508e+07,1474.133505,5.593094e+04,7.014495,21.970696,5.010267,0.484212,1.982115e+04,0.35142,0.260575
min,4.910000e+02,585.000000,4.033000e+03,3.600000,0.000000,0.000000,0.000000,0.000000e+00,0.00000,0.000000
25%,8.162000e+03,705.000000,4.468800e+04,13.500000,16.000000,8.000000,0.000000,5.905000e+03,0.00000,0.000000
50%,1.418300e+04,724.000000,6.175200e+04,16.900000,32.000000,10.000000,0.000000,1.102700e+04,0.00000,0.000000
75%,2.382200e+04,741.000000,8.688700e+04,21.700000,51.000000,14.000000,0.000000,1.933800e+04,0.00000,0.000000
max,1.000000e+08,7510.000000,8.713547e+06,70.500000,176.000000,76.000000,15.000000,1.730472e+06,7.00000,15.000000


## DATA CLEANING & PRE-PROCESSING

In [106]:
Null=loans.isnull().sum()
a=Null * 100/len(loans)
pd.DataFrame(a).sort_values(0,ascending=False)

,0
Months since last delinquent,53.104665
Credit Score,19.204911
Annual Income,19.204911
Years in current job,4.223856
Bankruptcies,0.205208
Tax Liens,0.009900
Years of Credit History,0.000000
Maximum Open Credit,0.000000
Current Credit Balance,0.000000
Number of Credit Problems,0.000000


In [107]:
# find number of duplicate values of each column/variable
for i in loans.columns:
    print(i,loans.duplicated(i).sum())

Loan ID 22197
Customer ID 22197
Loan Status 111105
Current Loan Amount 88566
Term 111105
Credit Score 110780
Years in current job 111095
Home Ownership 111103
Annual Income 73123
Purpose 111091
Monthly Debt 20745
Years of Credit History 110599
Months since last delinquent 110990
Number of Open Accounts 111055
Number of Credit Problems 111093
Current Credit Balance 77391
Maximum Open Credit 43798
Bankruptcies 111098
Tax Liens 111093


In [108]:
# find number of Unique values of each column/variable
for i in loans.columns:
    print(i,loans[i].unique().shape[0])

Loan ID 88910
Customer ID 88910
Loan Status 2
Current Loan Amount 22541
Term 2
Credit Score 327
Years in current job 12
Home Ownership 4
Annual Income 37984
Purpose 16
Monthly Debt 90362
Years of Credit History 508
Months since last delinquent 117
Number of Open Accounts 52
Number of Credit Problems 14
Current Credit Balance 33716
Maximum Open Credit 67309
Bankruptcies 9
Tax Liens 14


## Target Variable
### Loan Status 
#### A categorical variable indicating if the loan was paid back or defaulted.

#### Label encoding Loan Status 

In [109]:
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'Loan Status'.
loans['Loan Status']= label_encoder.fit_transform(loans['Loan Status'])

In [110]:
# Frequency Distribution
table = pd.crosstab(loans["Loan Status"],columns="Loan Status")
table["Percentage"] = pd.crosstab(loans["Loan Status"],columns="Term",normalize=True)*100
table
# Observation : Nearly 72% of the observations belongs to target class 'Fully Paid' whereas nearly 28% belongs to 'Charged Off'

col_0,Loan Status,Percentage
Loan Status,,
0,25173,22.656538
1,85934,77.343462


## Predictors

### Continous & Numerical Variables

### 01 & 02. Loan ID & Customer ID
#### loan ID is the unique identifier for your Affirm purchases.Customer ID is a unique identification code that you get from your bank.

##### Since loan ID and Customer ID are Nominal and is unique Identifier of each transactions and customers .so first we are removing duplicates from loan ID and removing both variable Loan ID and Customer ID

In [111]:
# removing duplicate data (Loan ID)
loans= loans.drop_duplicates(subset="Loan ID")

In [112]:
# Removing both Loan ID and Customer ID Variable
loans=loans.drop(['Loan ID','Customer ID'],axis=1)
loans.shape

(88910, 17)

### 03. Monthly Debt
#### The customer's monthly payment for their existing loans

##### The monthly debt column had values of object type.But when we tried to convert into 'float' type we encounterd an error due to the presence of '\\$' symbol.Hence we are removing every instance containing the '$' symbol with a blank character to enable type conversion into integer.

In [113]:
loans["Monthly Debt"].isnull().sum()

0

In [114]:
loans["Monthly Debt"].replace({'\$':''},regex=True,inplace=True) 

In [115]:
loans["Monthly Debt"].isnull().sum()

0

In [116]:
loans["Monthly Debt"]=pd.to_numeric(loans["Monthly Debt"])

### 04. Maximum Open Credit
#### The maximum credit limit for all credit sources

In [117]:
#errors = coerce automatically removes #VALUE!
loans["Maximum Open Credit"]=pd.to_numeric(loans["Maximum Open Credit"],errors='coerce') 

In [118]:
loans['Maximum Open Credit'].isnull().sum()

2

##### Imputing the null values with conditional median

In [119]:
loans_status_1=loans['Maximum Open Credit'][loans['Loan Status']==1]
loans_status_0=loans['Maximum Open Credit'][loans['Loan Status']==0]

In [120]:
loans_status_1.fillna(loans_status_1.median(),inplace=True)
loans_status_0.fillna(loans_status_0.median(),inplace=True)

In [121]:
loans['Maximum Open Credit'][loans['Loan Status']==1]=loans_status_1
loans['Maximum Open Credit'][loans['Loan Status']==0]=loans_status_0

In [122]:
loans['Maximum Open Credit'].isnull().sum()

0

### 05. Tax Liens
#### The number of tax liens.

In [ ]:
#loans['Tax Liens'].value_counts()

In [123]:
loans['Tax Liens'].isnull().sum()

8

##### Imputing the null values with conditional mode

In [124]:
print(loans[loans['Loan Status']=='Fully Paid']['Tax Liens'].mode())
print(loans[loans['Loan Status']=='Charged Off']['Tax Liens'].mode())

Series([], Name: Tax Liens, dtype: float64)
Series([], Name: Tax Liens, dtype: float64)


In [125]:
# Filling the missing values with the mode since it's a categorical column
loans['Tax Liens'].fillna(value=0.0,inplace=True)

In [126]:
loans['Tax Liens'].isnull().sum()

0

### 06. Months since last delinquent
#### Months since the last loan delinquent payment

In [ ]:
#loans['Months since last delinquent'].value_counts()

In [127]:
loans['Months since last delinquent'].isnull().sum()

48506

##### Imputing the null values with conditional median

In [128]:
loans_status_1=loans['Months since last delinquent'][loans['Loan Status']==1]
loans_status_0=loans['Months since last delinquent'][loans['Loan Status']==0]

In [129]:
loans_status_1.fillna(loans_status_1.median(),inplace=True)
loans_status_0.fillna(loans_status_0.median(),inplace=True)

In [130]:
loans['Months since last delinquent'][loans['Loan Status']==1]=loans_status_1
loans['Months since last delinquent'][loans['Loan Status']==0]=loans_status_0

In [131]:
loans['Months since last delinquent'].isnull().sum()

0

### 07. Number of Open Accounts
#### The total number of open credit cards

In [ ]:
#loans['Number of Open Accounts'].value_counts().sort_index(ascending=True)

In [132]:
loans['Number of Open Accounts'].isnull().sum()

0

### 08. Number of Credit Problems
#### The number of credit problems in the customer records.

In [ ]:
#loans['Number of Credit Problems'].value_counts()

In [133]:
loans['Number of Credit Problems'].isnull().sum()

0

### 09. Years of Credit History
#### The years since the first entry in the customers credit history

In [ ]:
#loans["Years of Credit History"].describe()

In [ ]:
# Note : There are customers with around 70 years of credit history
# loans['Years of Credit History'].value_counts().sort_index(ascending=False)

In [134]:
loans['Years of Credit History'].isnull().sum()

0

### 10. Annual Income
#### The customer's annual income

In [ ]:
#loans["Annual Income"].value_counts

In [135]:
loans['Annual Income'].isnull().sum()

21338

##### Imputing the null values with conditional median

In [136]:
loans_status_1=loans['Annual Income'][loans['Loan Status']==1]
loans_status_0=loans['Annual Income'][loans['Loan Status']==0]

In [137]:
loans_status_1.fillna(loans_status_1.median(),inplace=True)
loans_status_0.fillna(loans_status_0.median(),inplace=True)

In [138]:
loans['Annual Income'][loans['Loan Status']==1]=loans_status_1
loans['Annual Income'][loans['Loan Status']==0]=loans_status_0

In [139]:
loans['Annual Income'].isnull().sum()

0

### 11. Current Loan Amount
#### This is the loan amount that was either completely paid off, or the amount that was defaulted.

In [ ]:
#loans["Current Loan Amount"].value_counts

In [140]:
loans['Current Loan Amount'].isnull().sum()

0

### 12. Credit Score
#### A value between 0 and 800 indicating the riskiness of the borrowers credit history.credit score should be in  range of 300-850.

In [141]:
# The credit score as specified in the data dictionary are in the range of 0 to 800
# but some of the values are in range of 7000
loans['Credit Score'] = loans['Credit Score'].apply(lambda val: (val /10) if val>800 else val)
# loans['Credit Score'].value_counts

In [142]:
loans['Credit Score'].isnull().sum()

21338

##### Imputing the null values with conditional median

In [143]:
loans_status_1=loans['Credit Score'][loans['Loan Status']==1]
loans_status_0=loans['Credit Score'][loans['Loan Status']==0]
print(loans_status_1)

5044     739.0
5045     704.0
5046     751.0
5047     735.0
5048     745.0
         ...  
88904    730.0
88905    728.0
88906    710.0
88907    746.0
88909    718.0
Name: Credit Score, Length: 63737, dtype: float64


In [144]:
loans_status_1.fillna(loans_status_1.median(),inplace=True)
loans_status_0.fillna(loans_status_0.median(),inplace=True)

In [145]:
loans['Credit Score'][loans['Loan Status']==1]=loans_status_1
loans['Credit Score'][loans['Loan Status']==0]=loans_status_0

In [146]:
loans['Credit Score'].isnull().sum()

0

### 13. Current Credit Balance
#### The current balance (also called the credit card balance) reflects the current amount of all charges and payments made to your account up to that day

In [ ]:
# loans['Current Credit Balance'].value_counts

In [147]:
loans['Current Credit Balance'].isnull().sum()

0

### 14. Bankruptcies
#### The number of bankruptcies

In [ ]:
#loans["Bankruptcies"].value_counts()

In [148]:
loans['Bankruptcies'].isnull().sum()

192

##### Imputing the null values with conditional mode

In [149]:
print(loans[loans['Loan Status']=='Fully Paid']['Bankruptcies'].mode())
print(loans[loans['Loan Status']=='Charged Off']['Bankruptcies'].mode())

Series([], Name: Bankruptcies, dtype: float64)
Series([], Name: Bankruptcies, dtype: float64)


In [150]:
# Filling the missing values with the mode since it's a categorical column
loans['Bankruptcies'].fillna(value=0.0,inplace=True)

In [151]:
loans['Bankruptcies'].isnull().sum()

0

### Categorical Variables

### 15. Years in current job
#### A categorical variable indicating how many years the customer has been in their current job.

In [152]:
# Frequency Distribution
table = pd.crosstab(loans["Years in current job"],columns="Years in current job")
table["Percentage"] = pd.crosstab(loans["Years in current job"],columns="Years in current job",normalize=True)*100
table

col_0,Years in current job,Percentage
Years in current job,,
1 year,5765,6.774940
10+ years,27410,32.211815
2 years,8187,9.621238
3 years,7263,8.535367
4 years,5474,6.432962
5 years,6084,7.149824
6 years,5095,5.987567
7 years,4943,5.808938
8 years,4098,4.815907


In [153]:
# Convert string into Number
loans["Years in current job"]= loans["Years in current job"].replace({"< 1 year":0,"1 year":1,"2 years":2,"3 years":3,
                                                                      "4 years":4,"5 years":5,"6 years":6,"7 years":7,
                                                                      "8 years":8,"9 years":9,"10+ years":10}) 
#loans["Years in current job"]

In [154]:
loans['Years in current job'].isnull().sum()

3817

##### Imputing the null values with conditional mode

In [155]:
print(loans[loans['Loan Status']=='Fully Paid']['Years in current job'].mode())
print(loans[loans['Loan Status']=='Charged Off']['Years in current job'].mode())

Series([], Name: Years in current job, dtype: float64)
Series([], Name: Years in current job, dtype: float64)


In [156]:
# Filling the missing values with the mode since it's a categorical column
loans['Years in current job'].fillna(value=0.0,inplace=True)

In [158]:
loans["Years in current job"].isnull().sum()

0

### 16. Home Ownership
#### Categorical variable indicating home ownership. Values are "Rent", "Home Mortgage",and "Own". If the value is OWN, then the customer is a home owner with no mortgage

In [159]:
# Frequency Distribution
table = pd.crosstab(loans["Home Ownership"],columns="Home Ownership")
table["Percentage"] = pd.crosstab(loans["Home Ownership"],columns="Home Ownership",normalize=True)*100
table
# There is a Category called 'HaveMortgage' which is not available as per Description,so it must be
# Home Mortgage instead of HaveMortgage

col_0,Home Ownership,Percentage
Home Ownership,,
HaveMortgage,170,0.191205
Home Mortgage,43270,48.667192
Own Home,8075,9.082218
Rent,37395,42.059386


In [160]:
# Replacing every instance of 'HaveMortgage' with 'Home Mortgage'
loans['Home Ownership'] = loans['Home Ownership'].apply(lambda x: 'Home Mortgage' if x=='HaveMortgage' else x)

In [161]:
# Checking for the number of unique values in the Home Ownership series
loans['Home Ownership'].unique()

array(['Rent', 'Home Mortgage', 'Own Home'], dtype=object)

In [162]:
loans['Home Ownership'].isnull().sum()

0

### 17. Purpose
#### A description of the purpose of the loan.

In [163]:
# Frequency Distribution
table = pd.crosstab(loans["Purpose"],columns="Purpose")
table["Percentage"] = pd.crosstab(loans["Purpose"],columns="Purpose",normalize=True)*100
table

col_0,Purpose,Percentage
Purpose,,
Business Loan,1328,1.493645
Buy House,559,0.628726
Buy a Car,1196,1.345181
Debt Consolidation,70447,79.234057
Educational Expenses,92,0.103475
Home Improvements,5206,5.855359
Medical Bills,955,1.074120
Other,2850,3.205489
Take a Trip,454,0.510629


In [164]:
loans['Purpose'].nunique()

16

In [165]:
# replacing the instances of 'Other' with 'other'
loans['Purpose'].replace({'Other':'other'},inplace=True)

In [166]:
loans['Purpose'].nunique()

15

In [167]:
loans['Purpose'].isnull().sum()

0

### 18. Term
#### A categorical variable indicating if it is a short term or long term loan.

In [168]:
# Frequency Distribution
table = pd.crosstab(loans["Term"],columns="Term")
table["Percentage"] = pd.crosstab(loans["Term"],columns="Term",normalize=True)*100
table

col_0,Term,Percentage
Term,,
Long Term,21978,24.719379
Short Term,66932,75.280621


In [169]:
loans['Term'].isnull().sum()

0

### One hot encoding for Categorical variables (term,purpose,Home Ownership)

In [170]:
one_hot_encoded_data = pd.get_dummies(loans, columns = ['Term', 'Home Ownership','Purpose'])
Loans=one_hot_encoded_data
Loans

,Loan Status,Current Loan Amount,Credit Score,Years in current job,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,...,Purpose_Home Improvements,Purpose_Medical Bills,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding
0,0,12232,728.0,0.0,46643.0,777.39,18.0,10.0,12,0,...,0,0,0,0,0,0,0,0,0,0
1,0,25014,733.0,10.0,81099.0,892.09,26.7,31.0,14,0,...,0,0,0,0,0,0,0,0,0,0
2,0,16117,724.0,9.0,60438.0,1244.02,16.7,32.0,11,1,...,1,0,0,0,0,0,0,0,0,0
3,0,11716,740.0,3.0,34171.0,990.94,10.0,31.0,21,0,...,0,0,0,0,0,0,0,0,0,0
4,0,9789,686.0,10.0,47003.0,503.71,16.7,25.0,13,1,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88905,1,10136,728.0,10.0,35476.0,567.62,21.2,14.0,14,0,...,0,0,0,0,0,0,0,0,0,0
88906,1,29610,710.0,10.0,123375.0,2508.63,15.0,16.0,11,0,...,0,0,0,0,0,0,0,0,0,0
88907,1,24394,746.0,0.0,71148.0,818.20,36.4,9.0,8,0,...,0,0,0,0,0,0,0,0,0,0
88908,0,5030,722.0,0.0,25424.0,360.17,19.0,7.0,12,0,...,0,0,0,0,0,0,0,0,0,0


# Exploratory Data Analysis

### Uni-variate Analysis of Numerical Variable

In [171]:
EDA_Numerical = Loans.select_dtypes(exclude='object')

In [ ]:
# for col in EDA_Numerical.columns:
#     #histogram
#     EDA_Numerical.hist(column = col,
#               grid=False,        #Do not include the grid
#               figsize=(10,5),
#               color="pink",
#               edgecolor="black",
#               bins=20)     #(Length,Height) of the plot in inches
#     plt.show()
#     #boxplot
#     EDA_Numerical.boxplot(column= col,
#                 grid= False,
#                 figsize=(5,4),
#                 notch=True)
#     plt.show()

### Uni-variate Analysis of Categorical Variable

In [ ]:
# EDA_Categorical = Loans.select_dtypes(include='object')

In [ ]:
# for col in EDA_Categorical.columns:
#     Loans[col].value_counts().plot( kind='bar', xlabel=col, ylabel='frequency', color=['yellow','Orange','pink'])
#     plt.show()

### Predictors and Target

In [172]:
X_train = Loans.drop(["Loan Status"],axis=1)  
y_train = Loans["Loan Status"]

##### The Above Plot shows that our dataset is a class imbalanced dataset

# TEST DATA

In [ ]:
import os
os.getcwd()

In [173]:
labels= pd.read_csv(r'C:\Users\MY PC\Desktop\PPP\Loans - Public Test Set (IDs and Labels).csv')
nolabels= pd.read_csv(r'C:\Users\MY PC\Desktop\PPP\Loans - Public Test Set (No Labels).csv')
loans_test = pd.merge(left=labels,right=nolabels,how='inner',on='Loan ID')
loans_test.head()

,Loan ID,Loan Status_x,Customer ID,Loan Status_y,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,03e0da0c-fac3-42f8-a396-ed5a226466e0,Fully Paid,16ecb7be-c90f-4bdb-b168-da705d9b1fa7,To be predicted,23023,Long Term,597,10+ years,Home Mortgage,72230,Debt Consolidation,$613.96,14.0,NaN,11,0,23188,NaN,NaN,NaN
1,01825d5d-429a-4de6-b3d4-8618364d2df2,Charged Off,9e0609db-23c1-43b0-9326-0e67ca1e8f1d,To be predicted,8126,Short Term,696,2 years,Rent,55869,Debt Consolidation,$665.78,10.9,NaN,14,0,6135,NaN,NaN,NaN
2,c159a79a-8d2b-447c-92a9-052b5466d6cd,Fully Paid,c54e404f-1e94-45a2-9473-972e2f3c378b,To be predicted,5042,Short Term,745,10+ years,Home Mortgage,40332,Debt Consolidation,873.86,16.4,11.0,6,0,6698,17958.0,0.0,0.0
3,c1428058-2dc6-4a88-b64c-b6034dfb4f8f,Charged Off,37923e4e-df73-4093-a4b8-c2dfcbd0c8d8,To be predicted,14139,Long Term,677,< 1 year,Rent,70693,Debt Consolidation,482.48,19.5,33.0,8,0,5928,7273.0,0.0,0.0
4,20344186-79bd-487a-b7c4-f3c7b57fe3e8,Charged Off,d5dc29d5-06f9-452e-9d55-c1d543dca5dd,To be predicted,15191,Long Term,716,10+ years,Home Mortgage,38442,Debt Consolidation,611.87,17.8,0.0,4,0,10714,11243.0,0.0,0.0


In [174]:
# data type
loans_test.dtypes

Loan ID                          object
Loan Status_x                    object
Customer ID                      object
Loan Status_y                    object
Current Loan Amount               int64
Term                             object
Credit Score                      int64
Years in current job             object
Home Ownership                   object
Annual Income                     int64
Purpose                          object
Monthly Debt                     object
Years of Credit History         float64
Months since last delinquent    float64
Number of Open Accounts           int64
Number of Credit Problems         int64
Current Credit Balance            int64
Maximum Open Credit              object
Bankruptcies                    float64
Tax Liens                       float64
dtype: object

In [175]:
loans_test.shape

(44454, 20)

In [176]:
loans_test.describe()

,Current Loan Amount,Credit Score,Annual Income,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Bankruptcies,Tax Liens
count,44454.000000,44454.000000,4.445400e+04,44454.000000,44452.000000,44454.000000,44454.000000,44454.000000,44452.000000,44452.000000
mean,14000.404081,721.548702,7.262816e+04,18.299915,15.613403,11.158996,0.157556,15668.184258,0.110231,0.027738
std,8265.362190,27.562557,5.448082e+04,7.046328,22.571131,5.026281,0.468821,20278.741124,0.337787,0.254840
min,504.000000,585.000000,2.979000e+03,3.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7819.000000,710.000000,4.482350e+04,13.500000,0.000000,8.000000,0.000000,6005.250000,0.000000,0.000000
50%,12060.500000,731.000000,6.182550e+04,17.000000,0.000000,10.000000,0.000000,11214.000000,0.000000,0.000000
75%,19526.000000,741.000000,8.723350e+04,21.700000,28.000000,14.000000,0.000000,19501.000000,0.000000,0.000000
max,35875.000000,751.000000,5.001000e+06,61.500000,151.000000,50.000000,13.000000,975995.000000,7.000000,13.000000


## DATA CLEANING & PRE-PROCESSING

In [177]:
Null=loans_test.isnull().sum()
b=Null * 100/len(loans_test)
pd.DataFrame(b).sort_values(0,ascending=False)

,0
Years in current job,4.161605
Tax Liens,0.004499
Bankruptcies,0.004499
Maximum Open Credit,0.004499
Months since last delinquent,0.004499
Monthly Debt,0.000000
Current Credit Balance,0.000000
Number of Credit Problems,0.000000
Number of Open Accounts,0.000000
Years of Credit History,0.000000


In [178]:
# find number of duplicate values of each column/variable
for i in loans_test.columns:
    print(i,loans_test.duplicated(i).sum())

Loan ID 0
Loan Status_x 44452
Customer ID 0
Loan Status_y 44453
Current Loan Amount 26508
Term 44452
Credit Score 44287
Years in current job 44442
Home Ownership 44451
Annual Income 16690
Purpose 44438
Monthly Debt 5133
Years of Credit History 43958
Months since last delinquent 44349
Number of Open Accounts 44407
Number of Credit Problems 44442
Current Credit Balance 19876
Maximum Open Credit 9469
Bankruptcies 44445
Tax Liens 44441


In [179]:
# find number of Unique values of each column/variable
for i in loans_test.columns:
    print(i,loans_test[i].unique().shape[0])

Loan ID 44454
Loan Status_x 2
Customer ID 44454
Loan Status_y 1
Current Loan Amount 17946
Term 2
Credit Score 167
Years in current job 12
Home Ownership 3
Annual Income 27764
Purpose 16
Monthly Debt 39321
Years of Credit History 496
Months since last delinquent 105
Number of Open Accounts 47
Number of Credit Problems 12
Current Credit Balance 24578
Maximum Open Credit 34985
Bankruptcies 9
Tax Liens 13


## Target Variable
### Loan Status 
#### A categorical variable indicating if the loan was paid back or defaulted.

#### Label encoding Loan Status 

In [180]:
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'Loan Status'.
loans_test['Loan Status_x']= label_encoder.fit_transform(loans_test['Loan Status_x'])

In [181]:
# Frequency Distribution
table = pd.crosstab(loans_test["Loan Status_x"],columns="Loan Status_x")
table["Percentage"] = pd.crosstab(loans_test["Loan Status_x"],columns="Term",normalize=True)*100
table
# Observation : Nearly 72% of the observations belongs to target class 'Fully Paid' whereas nearly 28% belongs to 'Charged Off'

col_0,Loan Status_x,Percentage
Loan Status_x,,
0,12586,28.312413
1,31868,71.687587


## Predictors

### Continous & Numerical Variables

### 01 & 02. Loan ID & Customer ID
#### loan ID is the unique identifier for your Affirm purchases.Customer ID is a unique identification code that you get from your bank.

##### Since loan ID and Customer ID are Nominal and is unique Identifier of each transactions and customers .so first we are removing duplicates from loan ID and removing both variable Loan ID and Customer ID

In [182]:
# removing duplicate data (Loan ID)
loans_test= loans_test.drop_duplicates(subset="Loan ID")

In [183]:
# Removing both Loan ID and Customer ID Variable
loans_test=loans_test.drop(['Loan ID','Customer ID','Loan Status_y'],axis=1)
loans_test.shape

(44454, 17)

### 03. Monthly Debt
#### The customer's monthly payment for their existing loans

##### The monthly debt column had values of object type.But when we tried to convert into 'float' type we encounterd an error due to the presence of '\\$' symbol.Hence we are removing every instance containing the '$' symbol with a blank character to enable type conversion into integer.

In [184]:
loans_test["Monthly Debt"].isnull().sum()

0

In [185]:
loans_test["Monthly Debt"].replace({'\$':''},regex=True,inplace=True) 

In [188]:
loans_test["Monthly Debt"].replace({'\,':''},regex=True,inplace=True) 

In [189]:
loans_test["Monthly Debt"].isnull().sum()

0

In [190]:
loans_test["Monthly Debt"]=pd.to_numeric(loans_test["Monthly Debt"])

### 04. Maximum Open Credit
#### The maximum credit limit for all credit sources

In [191]:
#errors = coerce automatically removes #VALUE!
loans_test["Maximum Open Credit"]=pd.to_numeric(loans_test["Maximum Open Credit"],errors='coerce') 

In [192]:
loans_test['Maximum Open Credit'].isnull().sum()

3

##### Imputing the null values with conditional median

In [193]:
loans_status_1=loans_test['Maximum Open Credit'][loans_test['Loan Status_x']==1]
loans_status_0=loans_test['Maximum Open Credit'][loans_test['Loan Status_x']==0]

In [194]:
loans_status_1.fillna(loans_status_1.median(),inplace=True)
loans_status_0.fillna(loans_status_0.median(),inplace=True)

In [195]:
loans_test['Maximum Open Credit'][loans_test['Loan Status_x']==1]=loans_status_1
loans_test['Maximum Open Credit'][loans_test['Loan Status_x']==0]=loans_status_0

In [196]:
loans_test['Maximum Open Credit'].isnull().sum()

0

### 05. Tax Liens
#### The number of tax liens.

In [ ]:
#loans['Tax Liens'].value_counts()

In [197]:
loans_test['Tax Liens'].isnull().sum()

2

##### Imputing the null values with conditional mode

In [198]:
print(loans_test[loans_test['Loan Status_x']=='Fully Paid']['Tax Liens'].mode())
print(loans_test[loans_test['Loan Status_x']=='Charged Off']['Tax Liens'].mode())

Series([], Name: Tax Liens, dtype: float64)
Series([], Name: Tax Liens, dtype: float64)


In [199]:
# Filling the missing values with the mode since it's a categorical column
loans_test['Tax Liens'].fillna(value=0.0,inplace=True)

In [200]:
loans_test['Tax Liens'].isnull().sum()

0

### 06. Months since last delinquent
#### Months since the last loan delinquent payment

In [ ]:
#loans['Months since last delinquent'].value_counts()

In [201]:
loans_test['Months since last delinquent'].isnull().sum()

2

##### Imputing the null values with conditional median

In [202]:
loans_status_1=loans_test['Months since last delinquent'][loans_test['Loan Status_x']==1]
loans_status_0=loans_test['Months since last delinquent'][loans_test['Loan Status_x']==0]

In [203]:
loans_status_1.fillna(loans_status_1.median(),inplace=True)
loans_status_0.fillna(loans_status_0.median(),inplace=True)

In [204]:
loans_test['Months since last delinquent'][loans_test['Loan Status_x']==1]=loans_status_1
loans_test['Months since last delinquent'][loans_test['Loan Status_x']==0]=loans_status_0

In [205]:
loans_test['Months since last delinquent'].isnull().sum()

0

### 07. Number of Open Accounts
#### The total number of open credit cards

In [ ]:
#loans['Number of Open Accounts'].value_counts().sort_index(ascending=True)

In [206]:
loans_test['Number of Open Accounts'].isnull().sum()

0

### 08. Number of Credit Problems
#### The number of credit problems in the customer records.

In [ ]:
#loans['Number of Credit Problems'].value_counts()

In [207]:
loans_test['Number of Credit Problems'].isnull().sum()

0

### 09. Years of Credit History
#### The years since the first entry in the customers credit history

In [ ]:
#loans["Years of Credit History"].describe()

In [ ]:
# Note : There are customers with around 70 years of credit history
# loans['Years of Credit History'].value_counts().sort_index(ascending=False)

In [208]:
loans_test['Years of Credit History'].isnull().sum()

0

### 10. Annual Income
#### The customer's annual income

In [ ]:
#loans["Annual Income"].value_counts

In [209]:
loans_test['Annual Income'].isnull().sum()

0

##### Imputing the null values with conditional median

In [210]:
loans_status_1=loans_test['Annual Income'][loans_test['Loan Status_x']==1]
loans_status_0=loans_test['Annual Income'][loans_test['Loan Status_x']==0]

In [211]:
loans_status_1.fillna(loans_status_1.median(),inplace=True)
loans_status_0.fillna(loans_status_0.median(),inplace=True)

In [212]:
loans_test['Annual Income'][loans_test['Loan Status_x']==1]=loans_status_1
loans_test['Annual Income'][loans_test['Loan Status_x']==0]=loans_status_0

In [213]:
loans_test['Annual Income'].isnull().sum()

0

### 11. Current Loan Amount
#### This is the loan amount that was either completely paid off, or the amount that was defaulted.

In [ ]:
#loans["Current Loan Amount"].value_counts

In [214]:
loans['Current Loan Amount'].isnull().sum()

0

### 12. Credit Score
#### A value between 0 and 800 indicating the riskiness of the borrowers credit history.credit score should be in  range of 300-850.

In [215]:
# The credit score as specified in the data dictionary are in the range of 0 to 800
# but some of the values are in range of 7000
loans_test['Credit Score'] = loans_test['Credit Score'].apply(lambda val: (val /10) if val>800 else val)
# loans['Credit Score'].value_counts

In [216]:
loans_test['Credit Score'].isnull().sum()

0

### 13. Current Credit Balance
#### The current balance (also called the credit card balance) reflects the current amount of all charges and payments made to your account up to that day

In [ ]:
# loans['Current Credit Balance'].value_counts

In [217]:
loans_test['Current Credit Balance'].isnull().sum()

0

### 14. Bankruptcies
#### The number of bankruptcies

In [ ]:
#loans["Bankruptcies"].value_counts()

In [218]:
loans_test['Bankruptcies'].isnull().sum()

2

##### Imputing the null values with conditional mode

In [219]:
print(loans_test[loans_test['Loan Status_x']=='Fully Paid']['Bankruptcies'].mode())
print(loans_test[loans_test['Loan Status_x']=='Charged Off']['Bankruptcies'].mode())

Series([], Name: Bankruptcies, dtype: float64)
Series([], Name: Bankruptcies, dtype: float64)


In [220]:
# Filling the missing values with the mode since it's a categorical column
loans_test['Bankruptcies'].fillna(value=0.0,inplace=True)

In [221]:
loans_test['Bankruptcies'].isnull().sum()

0

### Categorical Variables

### 15. Years in current job
#### A categorical variable indicating how many years the customer has been in their current job.

In [222]:
# Frequency Distribution
table = pd.crosstab(loans_test["Years in current job"],columns="Years in current job")
table["Percentage"] = pd.crosstab(loans_test["Years in current job"],columns="Years in current job",normalize=True)*100
table

col_0,Years in current job,Percentage
Years in current job,,
1 year,3029,7.109661
10+ years,13806,32.405408
2 years,4175,9.799549
3 years,3485,8.179983
4 years,2717,6.377335
5 years,3039,7.133133
6 years,2502,5.872688
7 years,2322,5.450192
8 years,2147,5.039433


In [223]:
# Convert string into Number
loans_test["Years in current job"]= loans_test["Years in current job"].replace({"< 1 year":0,"1 year":1,"2 years":2,"3 years":3,
                                                                      "4 years":4,"5 years":5,"6 years":6,"7 years":7,
                                                                      "8 years":8,"9 years":9,"10+ years":10}) 
#loans["Years in current job"]

In [224]:
loans_test['Years in current job'].isnull().sum()

1850

##### Imputing the null values with conditional mode

In [225]:
print(loans_test[loans_test['Loan Status_x']=='Fully Paid']['Years in current job'].mode())
print(loans_test[loans_test['Loan Status_x']=='Charged Off']['Years in current job'].mode())

Series([], Name: Years in current job, dtype: float64)
Series([], Name: Years in current job, dtype: float64)


In [226]:
# Filling the missing values with the mode since it's a categorical column
loans_test['Years in current job'].fillna(value=0.0,inplace=True)

In [227]:
loans_test["Years in current job"].isnull().sum()

0

### 16. Home Ownership
#### Categorical variable indicating home ownership. Values are "Rent", "Home Mortgage",and "Own". If the value is OWN, then the customer is a home owner with no mortgage

In [228]:
# Frequency Distribution
table = pd.crosstab(loans_test["Home Ownership"],columns="Home Ownership")
table["Percentage"] = pd.crosstab(loans_test["Home Ownership"],columns="Home Ownership",normalize=True)*100
table
# There is a Category called 'HaveMortgage' which is not available as per Description,so it must be
# Home Mortgage instead of HaveMortgage

col_0,Home Ownership,Percentage
Home Ownership,,
Home Mortgage,21742,48.908985
Own Home,3892,8.755118
Rent,18820,42.335898


In [229]:
# Replacing every instance of 'HaveMortgage' with 'Home Mortgage'
loans_test['Home Ownership'] = loans_test['Home Ownership'].apply(lambda x: 'Home Mortgage' if x=='HaveMortgage' else x)

In [230]:
# Checking for the number of unique values in the Home Ownership series
loans_test['Home Ownership'].unique()

array(['Home Mortgage', 'Rent', 'Own Home'], dtype=object)

In [231]:
loans_test['Home Ownership'].isnull().sum()

0

### 17. Purpose
#### A description of the purpose of the loan.

In [232]:
# Frequency Distribution
table = pd.crosstab(loans_test["Purpose"],columns="Purpose")
table["Percentage"] = pd.crosstab(loans_test["Purpose"],columns="Purpose",normalize=True)*100
table

col_0,Purpose,Percentage
Purpose,,
Business Loan,656,1.475683
Buy House,290,0.652360
Buy a Car,535,1.203491
Debt Consolidation,35315,79.441670
Educational Expenses,52,0.116975
Home Improvements,2616,5.884735
Medical Bills,496,1.115760
Other,1395,3.138075
Take a Trip,234,0.526387


In [233]:
loans_test['Purpose'].nunique()

16

In [234]:
# replacing the instances of 'Other' with 'other'
loans_test['Purpose'].replace({'Other':'other'},inplace=True)

In [235]:
loans_test['Purpose'].nunique()

15

In [236]:
loans_test['Purpose'].isnull().sum()

0

### 18. Term
#### A categorical variable indicating if it is a short term or long term loan.

In [237]:
# Frequency Distribution
table = pd.crosstab(loans_test["Term"],columns="Term")
table["Percentage"] = pd.crosstab(loans_test["Term"],columns="Term",normalize=True)*100
table

col_0,Term,Percentage
Term,,
Long Term,10948,24.627705
Short Term,33506,75.372295


In [238]:
loans_test['Term'].isnull().sum()

0

### One hot encoding for Categorical variables (term,purpose,Home Ownership)

In [239]:
one_hot_encoded_data = pd.get_dummies(loans_test, columns = ['Term', 'Home Ownership','Purpose'])
Loans_Test=one_hot_encoded_data
Loans_Test

,Loan Status_x,Current Loan Amount,Credit Score,Years in current job,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,...,Purpose_Home Improvements,Purpose_Medical Bills,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding
0,1,23023,597,10.0,72230,613.96,14.0,0.0,11,0,...,0,0,0,0,0,0,0,0,0,0
1,0,8126,696,2.0,55869,665.78,10.9,0.0,14,0,...,0,0,0,0,0,0,0,0,0,0
2,1,5042,745,10.0,40332,873.86,16.4,11.0,6,0,...,0,0,0,0,0,0,0,0,0,0
3,0,14139,677,0.0,70693,482.48,19.5,33.0,8,0,...,0,0,0,0,0,0,0,0,0,0
4,0,15191,716,10.0,38442,611.87,17.8,0.0,4,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44449,0,12292,735,0.0,60506,1089.12,18.3,51.0,13,0,...,0,0,0,0,0,0,0,0,0,0
44450,1,4931,743,9.0,34514,509.07,13.0,0.0,8,1,...,0,0,0,0,0,0,0,0,0,0
44451,0,23272,632,10.0,68663,1224.49,15.5,0.0,16,0,...,0,0,0,0,0,0,0,0,0,0
44452,0,18611,668,10.0,44312,242.24,31.8,24.0,11,0,...,0,0,0,0,0,0,0,0,0,0


# Exploratory Data Analysis

### Uni-variate Analysis of Numerical Variable

In [ ]:
# EDA_Numerical = Loans_Test.select_dtypes(exclude='object')

In [ ]:
# for col in EDA_Numerical.columns:
#     #histogram
#     EDA_Numerical.hist(column = col,
#               grid=False,        #Do not include the grid
#               figsize=(10,5),
#               color="pink",
#               edgecolor="black",
#               bins=20)     #(Length,Height) of the plot in inches
#     plt.show()
#     #boxplot
#     EDA_Numerical.boxplot(column= col,
#                 grid= False,
#                 figsize=(5,4),
#                 notch=True)
#     plt.show()

### Uni-variate Analysis of Categorical Variable

In [ ]:
# EDA_Categorical = Loans.select_dtypes(include='object')

In [ ]:
# for col in EDA_Categorical.columns:
#     Loans[col].value_counts().plot( kind='bar', xlabel=col, ylabel='frequency', color=['yellow','Orange','pink'])
#     plt.show()

### Predictors and Target

In [244]:
X_test = Loans_Test.drop(["Loan Status_x"],axis=1)  
y_test = Loans_Test["Loan Status_x"]

##### The Above Plot shows that our dataset is a class imbalanced dataset

# MODEL FITTING

### LOGISTIC REGRESSION For Class Imbalanced Data

In [ ]:
logr = LogisticRegression()
logr.fit(X_train,y_train)
pred_test = logr.predict(X_test)   # predictions on transformed validation data

F1_score = f1_score((y_test),(pred_test))
accuracy = accuracy_score((y_test),(pred_test))
auc_roc=roc_auc_score((y_test),(pred_test))
Metric=pd.DataFrame({'Logistic Regression': [F1_score,accuracy,auc_roc]},
                    index=['F1_score','Accuracy','AUC-ROC'])
print(classification_report((y_test),(pred_test)))
Metric

In [ ]:
# X_train.columns

### DECISION TREE For Class Imbalanced Data

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
pred_test = dt.predict(X_test)   #predictions on transformed validation data

F1_score = f1_score((y_test),(pred_test))
accuracy = accuracy_score((y_test),(pred_test))
auc_roc=roc_auc_score((y_test),(pred_test))
Metric=pd.DataFrame({'Logistic Regression': [F1_score,accuracy,auc_roc]},
                    index=['F1_score','Accuracy','AUC-ROC'])
print(classification_report((y_test),(pred_test)))
Metric

### RANDOM FOREST For Class Imbalanced Data

In [247]:
RF = RandomForestClassifier()
RF.fit(X_train, y_train)
pred_test= RF.predict(X_test)
F1_score = f1_score((y_test),(pred_test))
accuracy = accuracy_score((y_test),(pred_test))
auc_roc=roc_auc_score((y_test),(pred_test))
Metric=pd.DataFrame({'Random Forest': [F1_score,accuracy,auc_roc]},
                    index=['F1_score','Accuracy','AUC-ROC'])
print(classification_report((y_test),(pred_test)))
Metric

              precision    recall  f1-score   support

           0       0.45      0.43      0.44     12586
           1       0.78      0.79      0.79     31868

    accuracy                           0.69     44454
   macro avg       0.61      0.61      0.61     44454
weighted avg       0.69      0.69      0.69     44454



,Random Forest
F1_score,0.786302
Accuracy,0.690332
AUC-ROC,0.610372


### Feature Selection using random forest

In [250]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train, y_train)
sel.get_support()
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)
print(selected_feat)

Index(['Credit Score', 'Annual Income', 'Months since last delinquent'], dtype='object')


### RANDOM FOREST For Class Imbalanced Data and Selected Features

In [ ]:
X_train=X_train[['Current Loan Amount', 'Credit Score', 'Annual Income', 'Monthly Debt',
       'Years of Credit History', 'Months since last delinquent',
       'Current Credit Balance', 'Maximum Open Credit']]
X_test=X_test[['Current Loan Amount', 'Credit Score', 'Annual Income', 'Monthly Debt',
       'Years of Credit History', 'Months since last delinquent',
       'Current Credit Balance', 'Maximum Open Credit']]

rf = RandomForestClassifier()
rf.fit(X_train,y_train)
pred_test = rf.predict(X_test)   #predictions on transformed validation data

F1_score = f1_score((y_test),(pred_test))
accuracy = accuracy_score((y_test),(pred_test))
auc_roc=roc_auc_score((y_test),(pred_test))
Metric=pd.DataFrame({'Random Forest For Class Imbalanced Data and Selected Features': [F1_score,accuracy,auc_roc]},
                    index=['F1_score','Accuracy','AUC-ROC'])
print(classification_report((y_test),(pred_test)))
Metric

# CLASS BALANCING USING SMOTE

In [252]:
counter=Counter(y_train)
sm = SMOTE(random_state = 2)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)
counter=Counter(y_train_sm)
print(counter)

Counter({0: 63737, 1: 63737})


### LOGISTIC REGRESSION For Class Balanced Data(Using SMOTE)

In [ ]:
logr = linear_model.LogisticRegression()
logr.fit(X_train_sm,y_train_sm)
pred_test = logr.predict(X_test)   #predictions on transformed validation data

F1_score = f1_score((y_test),(pred_test))
accuracy = accuracy_score((y_test),(pred_test))
auc_roc=roc_auc_score((y_test),(pred_test))
Metric=pd.DataFrame({'Logistic Regression with SMOTE': [F1_score,accuracy,auc_roc]},
                    index=['F1_score','Accuracy','AUC-ROC'])
print(classification_report((y_test),(pred_test)))
Metric

### RANDOM FOREST For Class Balanced Data(Using SMOTE)

In [254]:
RF = RandomForestClassifier()
RF.fit(X_train_sm, y_train_sm)
pred_test= RF.predict(X_test)

F1_score = f1_score((y_test),(pred_test))
accuracy = accuracy_score((y_test),(pred_test))
auc_roc=roc_auc_score((y_test),(pred_test))
Metric=pd.DataFrame({'Random Forest with SMOTE': [F1_score,accuracy,auc_roc]},
                    index=['F1_score','Accuracy','AUC-ROC'])
print(classification_report((y_test),(pred_test)))
Metric

              precision    recall  f1-score   support

           0       0.39      0.58      0.47     12586
           1       0.80      0.65      0.71     31868

    accuracy                           0.63     44454
   macro avg       0.60      0.61      0.59     44454
weighted avg       0.68      0.63      0.64     44454



,Random Forest with SMOTE
F1_score,0.713359
Accuracy,0.628065
AUC-ROC,0.614632


### DECISION TREE For Class Balanced Data(Using SMOTE)

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train_sm,y_train_sm)
pred_test = dt.predict(X_test)   #predictions on transformed validation data

F1_score = f1_score((y_test),(pred_test))
accuracy = accuracy_score((y_test),(pred_test))
auc_roc=roc_auc_score((y_test),(pred_test))
Metric=pd.DataFrame({'Decision Tree with SMOTE': [F1_score,accuracy,auc_roc]},
                    index=['F1_score','Accuracy','AUC-ROC'])
print(classification_report((y_test),(pred_test)))
Metric

### Feature Selection using random forest for class balanced data

In [256]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train_sm, y_train_sm)
sel.get_support()
selected_feat= X_train_sm.columns[(sel.get_support())]
len(selected_feat)
print(selected_feat)

Index(['Current Loan Amount', 'Credit Score', 'Months since last delinquent'], dtype='object')


### RANDOM FOREST For Class Balanced Data(Using SMOTE) and Selected Features

In [ ]:
X_train_sm=X_train_sm[['Credit Score', 'Monthly Debt','Months since last delinquent']]
X_test=X_test[['Credit Score', 'Monthly Debt','Months since last delinquent']]

rf = RandomForestClassifier()
rf.fit(X_train_sm,y_train_sm)
pred_test = rf.predict(X_test)   #predictions on transformed validation data

F1_score = f1_score((y_test),(pred_test))
accuracy = accuracy_score((y_test),(pred_test))
auc_roc=roc_auc_score((y_test),(pred_test))
Metric=pd.DataFrame({'Random Forest with Selected Features': [F1_score,accuracy,auc_roc]},
                    index=['F1_score','Accuracy','AUC-ROC'])
print(classification_report((y_test),(pred_test)))
Metric